In [12]:
import warnings
import logging
import os
import sys

# Suprimir logs de TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Redirigir salida estándar para suprimir warnings en Voila
warnings.filterwarnings('ignore')
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import io
from tensorflow.keras.applications.efficientnet import preprocess_input

In [14]:
# Cargar el modelo (reemplaza 'modelo_final.h5' por el nombre de tu modelo)
model = load_model('modelo/efficientnet_retina_v1_40_learn_3.0_batch_32_rep_3.h5')

In [15]:
# Widgets
upload_button = widgets.FileUpload(accept='.jpg, .jpeg', multiple=False)
output = widgets.Output()
message_label = widgets.Label(value="Por favor, sube una imagen en formato .jpg o .jpeg")
message_label2 = widgets.HTML(value="")
warning_label = widgets.HTML(value="<h3 style='color:#8e1414;'>Esto es solamente un prototipo que busca un acercamiento a la clasificación de enfermedades oculares que causan ceguera irreversible, especificamente: Glaucoma, Retinopatía diabética y Catarata; mediante imágenes de retinoscopía. Por lo tanto, las respuestas nunca serán 100% precisas.</h3>")
progress = widgets.FloatProgress(value=0, min=0, max=1, description='Progreso:',
                                 layout={'display': 'none'})  # Inicialmente oculta
# Cargar una imagen de la UDI
udi_image_path = 'imagen_udi/Logo-udi-web.png' 
with open(udi_image_path, "rb") as f:
    udi_image = f.read()
    
# Crear el widget para la imagen de la UDI
udi_image_widget = widgets.Image(
    value=udi_image,
    format='jpg',
    width=300,  # Ajusta el tamaño
    height=100
)

In [17]:
# Definir parámetros del modelo
img_size = (224, 224)
channels = 3  # o BGR o escala de grises
color = 'rgb'
img_shape = (img_size[0], img_size[1], channels)  # Forma esperada por el modelo

# Función para preprocesar la imagen
def preprocess_image(img):
    # Redimensionar la imagen
    img_resized = img.resize(img_size)
    
    # Convertir a array numpy
    img_array = np.array(img_resized)
    img_array = preprocess_input(img_array)
    # Verificar que la forma sea correcta y normalizar si es necesario
    if img_array.shape != img_shape:
        raise ValueError(f"La forma de la imagen procesada {img_array.shape} no coincide con {img_shape}")
    
    # Añadir dimensión batch
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

# Función para procesar la imagen y realizar la predicción
def process_image(change):
    output.clear_output()
    progress.value = 0.0
    progress.layout.display = 'none'  # Ocultar barra de progreso inicialmente
    message_label2.value = ""  # Reiniciar mensaje

    if upload_button.value:
        progress.layout.display = 'flex'
        progress.value = 0.1

        uploaded_file = upload_button.value[0]
        file_name = uploaded_file.get('name', '')  # Obtener nombre del archivo
        content = uploaded_file.get('content', b'')

        # Validar si el archivo es una imagen en formato .jpg o .jpeg
        if not file_name.lower().endswith(('.jpg', '.jpeg')):
            message_label2.value = f"<h2 style='color:red;'>Error: El archivo {file_name} no es una imagen válida. Sube un archivo .jpg o .jpeg</h2>"
            progress.layout.display = 'none'
            upload_button.value = None  # Limpiar contenido cargado
            return

        try:
            # Intentar abrir el archivo como una imagen
            img = Image.open(io.BytesIO(content.tobytes()))
        except Exception as e:
            message_label2.value = f"<h2 style='color:red;'>Error: No se pudo abrir el archivo {file_name}. Asegúrate de que sea una imagen válida.</h2>"
            progress.layout.display = 'none'
            upload_button.value = None  # Limpiar contenido cargado
            return

        # Mostrar la imagen cargada
        with output:
            plt.imshow(img)
            plt.axis('off')
            plt.show()

        progress.value = 0.5

        try:
            # Preprocesar la imagen usando la función definida
            img_array = preprocess_image(img)

            # Realizar la predicción
            prediction = model.predict(img_array)
            predicted_class = np.argmax(prediction, axis=1)

            progress.value = 0.8

            class_names = {0: "Catarata", 1: "Retinopatía Diabética", 2: "Glaucoma", 3: "Normal"}
            predicted_label = class_names.get(predicted_class[0], "Clase desconocida")

            message_label2.value = f"<h2 style='color:#046c9c;'>Predicción del modelo: {predicted_label}</h2>"
        except ValueError as ve:
            message_label2.value = f"<h2 style='color:red;'>Error: {str(ve)}</h2>"
        except Exception as e:
            message_label2.value = f"<h2 style='color:red;'>Error inesperado: {str(e)}</h2>"
        finally:
            progress.value = 1.0
    else:
        message_label2.value = "<h2 style='color:red;'>Error: Por favor sube una imagen válida en formato .jpg o .jpeg</h2>"
    
    # Limpiar siempre el contenido cargado
    upload_button.value = None


In [ ]:
# Asociar la función al cambio en el botón de carga
upload_button.observe(process_image, names='value')

# Mostrar los widgets 
display(udi_image_widget)  # Imagen de la UDI
display(widgets.HTML("<h1>Clasificación de imágenes</h2>"))
display(warning_label)  # Advertencia de prototipo
display(message_label)
display(upload_button)
display(output)  # Aquí se mostrará la imagen
display(progress)  # Barra de progreso
display(message_label2)  # Aquí se mostrará la predicción